In [4]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from dotenv import load_dotenv

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Database part

In [6]:
loader = WebBaseLoader(
    web_paths=("https://medium.com/@kbdhunga/advanced-rag-multi-query-retriever-approach-ad8cd0ea0f5b",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#    ),
)
blog_docs = loader.load()

In [7]:
# Spliting 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=50,
    chunk_overlap=10)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=embeddings)

c:\Users\IVR\github\ai_venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


create a retriever

In [9]:
retriever = vectorstore.as_retriever()

# Few Short Prompting

Few Shot Examples

In [1]:

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [3]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import os

generate_queries_step_back = prompt | ChatGroq(temperature=0,api_key=os.getenv('groq_api_key')) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'how do LLM agents break down tasks? or what is the process of dividing tasks for LLM agents? or how are tasks handled by LLM agents?'

Response Prompt

In [11]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGroq(temperature=0,api_key=os.getenv('groq_api_key'))
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for Language Model (LLM) agents refers to the process of breaking down a complex task into smaller, manageable sub-tasks that can be solved independently. This approach is particularly useful when dealing with complex queries or tasks that require a deep understanding of language and context.\n\nIn the context of Retrieval-Augmented Generation (RAG), task decomposition is achieved through a two-step process. First, the query is transformed into an embedding vector, which is then used to perform a similarity search in a database of document vectors. This step helps to retrieve the most relevant documents related to the query.\n\nNext, the RAG system merges the content of the retrieved documents with the original query to form a comprehensive dataset. This dataset is then processed by a LLM model, which produces a contextually relevant response to the query.\n\nThe advantage of this approach is that it allows the LLM agent to harness the power of diversity by retrievi